# Face Recognition using KNN
In this notebook, we aim to apply KNN based on cosine similarity distances from InceptionV3 and distance metrics from DeepFace implementations as the final culmination of the semester project.

In [1]:
# imports
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os
import cv2
from ast import literal_eval

import keras
import tensorflow as tf

In [2]:
from deepface import DeepFace